In [ ]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters langchain-chroma langchain-core langchain-upstage

#### Chunk 단위로 데이터를 Chunking 하는 이유
- split 된 데이터를 LLM에 전달함으로써 Input Token 절약
- 입력 토큰이 적을수록 LLM이 첫 글자를 생성하는 속도가 빨라짐

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # Chunk 한개가 가질 수 있는 토큰 수
    chunk_overlap=300, # Chunk 를 겹치게 자르는, 1 - 3, 다음은 2 - 4 이런식으로
)

# 문서를 가져와서,
loader = Docx2txtLoader('./tax.docx')

# Chunk 단위로 문서를 쪼갠다
document_list = loader.load_and_split(
    text_splitter=text_splitter
)

In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_upstage import UpstageEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

upstage_embedding = UpstageEmbeddings(
    model="solar-embedding-1-large"
)

- Chroma : Vector in-memory Database

In [ ]:
from langchain_chroma import Chroma

# 위에서 만든 Embedding 으로 우리가 만든 document를 데이터베이스에 저장한다. collection_name : Chroma 의 테이블 이름,  : 영속시킬 폴더, 인메모리 -> 파일 기반으로
database = Chroma.from_documents(documents = document_list, embedding = upstage_embedding, collection_name='chroma-tax', persist_directory='./chroma')
#database = Chroma(collection_name='chroma-tax', persist_directory='./chroma', embedding_function=embedding)

In [ ]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrived_docs = database.similarity_search(query, k = 10)
print(retrived_docs)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [Context] 를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrived_docs}

Question : {query}
"""

ai_message = llm.invoke(prompt)

In [ ]:
ai_message.content

In [ ]:
# Retrival Chain
%pip install -U langchain langchainhub --quiet

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs={"prompt" : prompt}
)

ai_message = qa_chain.invoke({"query" : query})

In [ ]:
ai_message